In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

pd.options.display.max_columns = 99

In [2]:
df = dict()

df['scoring'] = pd.read_excel('../output.xlsx', index_col=[0,1])
df['tr'] = pd.read_csv('../dunnhumbyGrocery/transaction_data.csv')
df['prod'] = pd.read_csv('../dunnhumbyGrocery/product.csv')
df['demo'] = pd.read_csv('../dunnhumbyGrocery/hh_demographic.csv')
df['campDesc'] = pd.read_csv('../dunnhumbyGrocery/campaign_desc.csv')
df['campData'] = pd.read_csv('../dunnhumbyGrocery/campaign_table.csv')

In [3]:
df['scoring'] = df['scoring'].droplevel(0)
df['scoring'].reset_index(inplace=True)

In [4]:
df['scoring'] = df['scoring'].rename(columns={'index':'SUB_COMMODITY_DESC', 0:'score'})

In [5]:
dfall = df['scoring'].merge(df['prod'], on = 'SUB_COMMODITY_DESC', how='right')
dfall = dfall.merge(df['tr'], on = 'PRODUCT_ID')
print(len(dfall))
dfall = dfall.merge(df['demo'], on = 'household_key')
print(len(dfall))

2918921
1602782


In [6]:
#pctOff = np.arange(0.1, 0.5, 0.044)
pctOff = np.arange(0.1, 0.4, 0.033)
dfall['Score'].replace([1,2,3,4,5,6,7,8,9,10],pctOff, inplace=True)
dfall['DISC'] = dfall['Score'].multiply(dfall['SALES_VALUE'], fill_value=0)

In [7]:
#GET ALL UNIQUE HOUSEHOLD KEYS, GROUP EACH HOUSEHOLD KEY BY DAY

houseKeys = dfall.household_key.unique().tolist()
ghDic = {}
ghGby = {}
hk100 = []
for x in houseKeys:
    ghDic['houseKey_' + str(x)] = dfall.loc[dfall['household_key'] == x]
    ghGby['houseKey_' + str(x)] = ghDic['houseKey_' + str(x)].groupby(['DAY'])
    if len(ghGby['houseKey_' + str(x)]) > 100:
        hk100.append(x)

In [8]:
#RETRIEVE EACH GROUP, INDEXED BY HOUSEHOLD KEY AND DAY
group100 = {}
for housekey in hk100:
    group100[housekey] = {}
    for day in ghGby['houseKey_' + str(housekey)].DAY.unique().keys():
        group100[housekey][day] = ghGby['houseKey_' + str(housekey)].get_group(day)

In [9]:
houseScore = {}
for x in group100.keys():
    houseScore[x] = {}
    for y in group100[x]:
        houseScore[x][y] = group100[x][y]['DISC'].sum(axis=0)

In [10]:
houseTot = {}
houseAvg = {}
lenD = {}
for x in houseScore.keys():
    houseTot[x] = sum(houseScore[x].values())
    houseAvg[x] = int(sum(houseScore[x].values())/len(houseScore[x].values()))
    lenD[x] = len(houseScore[x].values())
    print(x, houseAvg[x],houseTot[x], lenD[x])

1228 4 2177.77784 469
358 5 1644.3416700000014 307
908 6 1113.5657399999998 178
596 4 438.22022000000015 104
1749 2 272.6163200000001 115
574 10 1165.9020000000007 112
1421 3 595.5671700000001 195
17 5 552.58793 101
2062 5 1019.6655500000006 174
1452 0 230.7453000000001 267
979 17 2947.435530000001 172
2116 10 2033.34847 185
2260 6 755.29417 117
595 2 346.6945899999999 128
900 3 1577.92973 521
409 5 686.6852900000004 121
404 3 739.2828799999997 223
1609 8 2656.9827000000005 327
1264 7 860.8521600000003 118
209 6 878.4902300000002 143
1130 4 1048.9762900000003 249
2378 7 1083.2188699999997 150
715 4 588.7537100000001 134
2154 4 440.31166000000013 106
1720 5 793.2078599999998 139
1595 5 688.6560800000001 122
831 9 1471.61152 152
560 3 988.5960200000005 258
1762 5 1557.3322000000005 275
712 8 1867.7292700000003 226
588 4 977.2774899999996 211
2488 8 939.1895099999999 108
1517 3 477.57880000000006 120
67 5 1105.0400000000002 189
576 4 593.5116799999998 138
1169 11 1135.7623100000005 103
11

1391 9 1332.6365199999987 148
800 4 1867.6233799999982 379
2133 6 803.9220200000001 121
2041 11 1432.0387000000003 126
1797 13 1439.6452200000003 106
218 9 1277.61378 137
1352 7 872.2306799999999 119
1020 12 1850.7070899999985 153
699 9 1215.6818700000001 124
85 3 864.7273099999999 245
2284 12 2780.0785300000002 221
2343 2 497.08972 196
1896 5 616.5849499999999 112
2252 5 1338.1459400000006 227
1261 2 426.57593000000014 180
1831 6 969.7865899999998 144
883 5 1463.64548 259
18 7 1019.19307 132
859 4 790.6651999999996 171
49 9 995.57939 108
1256 8 1406.9873599999999 157
1791 4 620.12023 125
1247 11 1277.6593500000001 111
740 12 1304.75572 108
806 10 1128.1416900000002 104
1419 3 1379.8091100000008 388
1710 13 1972.09717 151
1928 2 231.73558999999997 111
334 2 755.2333799999999 278
1337 7 1491.293810000001 211
2111 3 388.6442599999999 114
2455 7 720.4124499999999 102
817 5 710.4033200000003 140
1935 9 1343.9249200000006 147
1726 8 1111.66738 138
2317 14 1944.2126699999994 136
1914 1 279.3

In [11]:
disctot = sum(houseTot.values())
print(disctot)
print(len(houseKeys))

519824.29108
801


In [13]:
totSum = disctot / len(houseKeys)
print(totSum)

648.9691524094882


648*165501325 = 107,244,858,600
It would cost 107 billion to give every american healthier eating choices.